In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os
import time
import numpy as np

In [3]:
import pydrake
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions
from pydrake.common import FindResourceOrThrow
from pydrake.geometry import SceneGraph
from pydrake.math import RigidTransform, RollPitchYaw
from pydrake.multibody.optimization import CalcGridPointsOptions, Toppra
from pydrake.multibody.parsing import LoadModelDirectives, Parser, ProcessModelDirectives
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.tree import RevoluteJoint
from pydrake.solvers.mathematicalprogram import MathematicalProgram, Solve
from pydrake.solvers.mosek import MosekSolver
from pydrake.systems.analysis import Simulator
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.primitives import TrajectorySource
from pydrake.trajectories import PiecewisePolynomial
from pydrake.all import Variable
from pydrake.all import MultibodyPositionToGeometryPose, ConnectMeshcatVisualizer, Role, Sphere
from pydrake.all import (
    ConvexSet, HPolyhedron, Hyperellipsoid,
    MathematicalProgram, Solve, le, IpoptSolver,
    Role, Sphere,
    Iris, IrisOptions, MakeIrisObstacles, Variable
)
from pydrake.all import (
    eq, SnoptSolver,
    Sphere, Ellipsoid, GeometrySet,
    RigidBody_, AutoDiffXd, initializeAutoDiff,
)

import pydrake.symbolic as sym

from meshcat import Visualizer

# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

def xreplace(expr, rules):
        if isinstance(expr, float) or isinstance(expr, sym.Variable):
            return expr
        assert isinstance(expr, sym.Expression), expr
        for old, new in rules:
            if expr.EqualTo(old):
                return new
        ctor, old_args = expr.Deconstruct()
        new_args = [xreplace(e, rules) for e in old_args]
        return ctor(*new_args)

In [4]:
from pydrake.all import RationalFunction
print("success")

success


In [4]:
simple_collision = True
# gripper_welded = True

vis = Visualizer(zmq_url=zmq_url)
vis.delete()
display(vis.jupyter_cell())

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
parser.package_map().Add( "wsg_50_description", os.path.dirname(FindResourceOrThrow(
            "drake/manipulation/models/wsg_50_description/package.xml")))

directives_file = FindResourceOrThrow("drake/sandbox/planar_iiwa_simple_collision_welded_gripper.yaml") \
    if simple_collision else FindResourceOrThrow("drake/sandbox/planar_iiwa_dense_collision_welded_gripper.yaml")
directives = LoadModelDirectives(directives_file)
models = ProcessModelDirectives(directives, plant, parser)

q0 = [-0.2, -1.2, 1.6]
index = 0
for joint_index in plant.GetJointIndices(models[0].model_instance):
    joint = plant.get_mutable_joint(joint_index)
    if isinstance(joint, RevoluteJoint):
        joint.set_default_angle(q0[index])
        index += 1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False)

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/
Connected to meshcat-server.


In [5]:
seed_points = np.array([[0.0, -2.016, 1.975], # in tight
                        [-1, -2, 0.5],        # neutral pose
                        [0.3, -0.8, 0.5],     # above shelf
                        [0.25, -1.6, -0.25],  # in shelf 1
                        [0.07, -1.8, -0.2],   # leaving shelf 1
                        [-0.1, -2, -0.3]])    # out of shelf 1


from pydrake.all import InverseKinematics
cspace_seed_points = []
hand_body = plant.GetBodyByName("iiwa_link_ee_kuka")
hand_frame = hand_body.body_frame()
q0 = plant.GetPositions(plant_context)
for i in range(seed_points.shape[0]):
    ik = InverseKinematics(plant, plant_context)
    ik.AddMinimumDistanceConstraint(0.001, 0.01)
    ik.AddPositionConstraint(hand_frame, [0, 0, 0], plant.world_frame(), seed_points[i,:], seed_points[i,:])
    ik.prog().SetInitialGuess(ik.q(), q0)
    result = Solve(ik.prog())
    if not result.is_success():
        print(seed_points[i,:])
#         assert result.is_success()
    cspace_seed_points.append(result.GetSolution(ik.q()))
cspace_seed_points = np.array(cspace_seed_points)
print(cspace_seed_points)

# seed_points = np.array([[0.0, 0.0, 0.0, 0.0,0.0, 0.0]])    # out of shelf 1

# traj = PiecewisePolynomial.FirstOrderHold(np.array([0, 1]), np.array([seed_points[4], seed_points[1]]).T)
# visualize_trajectory(traj)

[ 0.    -2.016  1.975]
[-1.  -2.   0.5]
[ 0.3 -0.8  0.5]
[ 0.25 -1.6  -0.25]
[ 0.07 -1.8  -0.2 ]
[-0.1 -2.  -0.3]
[[ 8.55974367e-07  3.51284882e-06  8.23286705e-06]
 [-1.42225948e+00 -2.49953376e-07  1.22251762e-06]
 [ 1.78716687e-01 -1.96024646e+00  2.09439510e+00]
 [ 2.02760502e+00 -1.12246694e+00  1.36453888e+00]
 [-7.56258558e-01 -2.09439510e+00  2.05217654e+00]
 [-1.36432306e+00 -2.09439510e+00  2.05217079e+00]]


In [6]:
def MakeFromSceneGraph(query, geom, expressed_in=None):
    shape = query.inspector().GetShape(geom)
    if isinstance(shape, (Sphere, Ellipsoid)):
        return Hyperellipsoid(query, geom, expressed_in)
    return HPolyhedron(query, geom, expressed_in)

In [7]:
def generate_rationalize_trig_expr_rules(q_var, t_var):
    rules = []
    denom = 1
    for i in range(t.shape[0]):
        sin_rule = (sym.sin(q[i]), (2*t[i])/(1+t[i]**2))
        cos_rule = (sym.cos(q[i]), (1-t[i]**2)/(1+t[i]**2))
        rules += [sin_rule, cos_rule]
        denom *= (1+t[i]**2)
    return rules, denom


def rationalize_trig_expr(expr, rules):
    return xreplace(expr, rules)

In [8]:
def rationalize_iris_equality_constraint(p_WA, p_WB, trig_rules, trig_denom):
    #TODO need to clear denominators see later
    lhs_WA = [None for i in range(p_WA.shape[0])]
    rhs_WB = [None for i in range(p_WA.shape[0])]
    for i in range(p_WA.shape[0]):
        lhs_WA[i] = xreplace(p_WA[i], rules)*trig_denom
        rhs_WB[i] = xreplace(p_WB[i], rules)*trig_denom
    return lhs_WA, rhs_WB



In [9]:
def FindClosestCollision(sym_plant, sym_context, frameA, frameB, setA, setB, E, Gt, ht):
    prog = MathematicalProgram()
    #mismatch in shape of t and E.
    q = sym.MakeVectorContinuousVariable(3,"q")
    t = prog.NewContinuousVariables(Gt.shape[1], "t")
    trig_rationalize_rules, trig_denoms = generate_rationalize_trig_expr_rules(q,t)

    p_AA = prog.NewContinuousVariables(3, "p_AA")
    p_BB = prog.NewContinuousVariables(3, "p_BB")
    setA.AddPointInSetConstraints(prog, p_AA)
    setB.AddPointInSetConstraints(prog, p_BB)

    prog.AddQuadraticErrorCost(E.A().T @ E.A(), E.center(), t)

    sym_plant.SetPositions(sym_context, q)
    X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, frameA)
    X_WB = sym_plant.EvalBodyPoseInWorld(sym_context, frameB)
    print(X_WB)
    # Add +0 pending https://github.com/RobotLocomotion/drake/issues/15216
    p_WA = X_WA.multiply(p_AA+0)
    p_WB = X_WB.multiply(p_BB+0)
    #add equality constraint as SOS constraint
#     result = snopt.Solve(prog)


In [10]:
def IrisInConfigSpace(plant, plant_context, sample, iris_options):
    #reimplement IrisInConfigurationSpace method in Python so I can tweak grow ellipse
    nq = plant.num_positions()
    P = HPolyhedron(plant.GetPositionLowerLimits(), plant.GetPositionUpperLimits())
    kEpsilonEllipsoid = 1e-2
    E = Hyperellipsoid.MakeHypersphere(kEpsilonEllipsoid, sample)
    query_object = plant.get_geometry_query_input_port().Eval(plant_context)
    inspector = query_object.inspector()

    #Skipping IrisConvex set maker steps? Should be IrisConvexSetMaker
    pairs = inspector.GetCollisionCandidates()
    N = len(pairs)
    geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
    sets = {geom:MakeFromSceneGraph(query_object, geom, inspector.GetFrameId(geom)) for geom in geom_ids}

    #Symbolic plants and bodies
    sym_plant = plant.ToSymbolic()
    sym_context = sym_plant.CreateDefaultContext()
    sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}

    #preallocated
    Gt = np.zeros((P.A().shape[0]+2*N, nq))
    ht = np.zeros(P.A().shape[0]+2*N)

    best_volume = E.Volume()
    iteration = 0

    #ignoring solver logic?
    i = 0
#     while True:
    while i < 1:
        num_constraints = 2*nq
        # Find separating hyperplanes


        for (geomA, geomB) in pairs:
            print(f"geomA={inspector.GetName(geomA)}, geomB={inspector.GetName(geomB)}")
            collision  = FindClosestCollision(sym_plant, sym_context, sym_bodies[geomA], sym_bodies[geomB],
                                              sets[geomA], sets[geomB], E,
                                              Gt[:num_constraints, :], ht[:num_constraints])
            #only do one
            break
        i+=1


In [11]:
# iris_options = IrisOptions()
# iris_options.require_sample_point_is_contained = True
# iris_options.iteration_limit = 10
# iris_options.enable_ibex = False

# regions = []
# for i in range(seed_points.shape[0]):
#     start_time = time.time()
# #     hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
#     hpoly = IrisInConfigSpace(plant, plant_context, seed_points[i,:], iris_options)
#     ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
#     print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
#           ellipse.center(), flush=True)
#     regions.append(hpoly)


In [12]:
#unpack a symbolic geometry
query_object = plant.get_geometry_query_input_port().Eval(plant_context)
inspector = query_object.inspector()
pairs = inspector.GetCollisionCandidates()
sym_plant = plant.ToSymbolic()
sym_context = sym_plant.CreateDefaultContext()
geom_ids = inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)
sym_bodies = {geom:sym_plant.GetBodyFromFrameId(inspector.GetFrameId(geom)) for geom in geom_ids}
geomA = iter(pairs).__next__()[0]
frameA = sym_bodies[geomA]
prog = MathematicalProgram()

q = sym.MakeVectorContinuousVariable(3,"q")
t = prog.NewContinuousVariables(q.shape[0], "t")

sym_plant.SetPositions(sym_context, q)
X_WA = sym_plant.EvalBodyPoseInWorld(sym_context, frameA)
p_AA = prog.NewContinuousVariables(3, "p_AA")
p_WA = X_WA.multiply(p_AA+0)

for i in range(p_WA.shape[0]):
    p_WA[i] = p_WA[i].Expand()
print(type(p_WA))

<class 'numpy.ndarray'>


In [13]:
# print(p_WA[0])
tmp = p_WA[0]
# print(X_WA.rotation()[0,0].GetVariables())
print(tmp)

(-2.0208057374588098e-32 + 1.2246467991473532e-16 * p_AA(0) - 5.9965918743193009e-28 * p_AA(1) - 2.2958363950258565e-43 * p_AA(2) + 7.6571373978538989e-16 * (p_AA(0) * sin(q(0))) + 1.2246467991473532e-16 * (p_AA(0) * sin(q(0)) * sin(q(1))) - 5.9965918743192982e-28 * (p_AA(0) * sin(q(0)) * sin(q(1)) * sin(q(2))) - 4.8965888601467475e-12 * (p_AA(0) * sin(q(0)) * sin(q(1)) * cos(q(2))) + 4.8965888601467475e-12 * (p_AA(0) * sin(q(0)) * sin(q(2)) * cos(q(1))) - 5.9965918743192982e-28 * (p_AA(0) * sin(q(0)) * cos(q(1)) * cos(q(2))) + 4.591672790051713e-43 * (p_AA(0) * sin(q(0)) * cos(q(2))) - 1.1483866719617346e-47 * (p_AA(0) * sin(q(1))) - 4.8965888601467475e-12 * (p_AA(0) * sin(q(1)) * sin(q(2)) * cos(q(0))) + 5.9965918743192982e-28 * (p_AA(0) * sin(q(1)) * cos(q(0)) * cos(q(2))) + 4.591672790051713e-43 * (p_AA(0) * sin(q(1)) * cos(q(2))) - 5.9965918743192982e-28 * (p_AA(0) * sin(q(2)) * cos(q(0)) * cos(q(1))) - 4.591672790051713e-43 * (p_AA(0) * sin(q(2)) * cos(q(1))) - 9.3772888049132665

In [23]:
def generate_rationalize_trig_expr_rules(q_var, t_var):
    rules = []
    for i in range(t.shape[0]):
        sin_rule = (sym.sin(q[i]), (2*t[i])/(1+t[i]**2))
        cos_rule = (sym.cos(q[i]), (1-t[i]**2)/(1+t[i]**2))
        rules += [sin_rule, cos_rule]
    return rules

rules = generate_rationalize_trig_expr_rules(q, t)
rules_denoms = 1
for i in range(t.shape[0]):
    rules_denoms *= (1+t[i]**2)

def rationalize_trig_expr(expr):
    return xreplace(expr, rules)

expr_in_q = p_WA[0]
# print(expr_in_q)
print()
expr_in_t = rationalize_trig_expr(expr_in_q)
tmp = (rules_denoms*expr_in_t).Expand()
#this is the type of expr we expect. Now need to figure out how to clear denominators
print(tmp)


(-2.0208057374588098e-32 + 1.2246467991473532e-16 * p_AA(0) - 5.9965918743193009e-28 * p_AA(1) - 2.2958363950258565e-43 * p_AA(2) - 2.0208057374588098e-32 * (pow(t(0), 2) * pow(t(1), 2)) - 2.0208057374588098e-32 * (pow(t(0), 2) * pow(t(1), 2) * pow(t(2), 2)) + 1.2246467991473532e-16 * (pow(t(0), 2) * pow(t(1), 2) * pow(t(2), 2) * p_AA(0)) - 5.9965918743193009e-28 * (pow(t(0), 2) * pow(t(1), 2) * pow(t(2), 2) * p_AA(1)) - 2.2958363950258565e-43 * (pow(t(0), 2) * pow(t(1), 2) * pow(t(2), 2) * p_AA(2)) + 1.2246467991473532e-16 * (pow(t(0), 2) * pow(t(1), 2) * p_AA(0)) - 5.9965918743193009e-28 * (pow(t(0), 2) * pow(t(1), 2) * p_AA(1)) - 2.2958363950258565e-43 * (pow(t(0), 2) * pow(t(1), 2) * p_AA(2)) - 2.0208057374588098e-32 * (pow(t(0), 2) * pow(t(2), 2)) + 1.2246467991473532e-16 * (pow(t(0), 2) * pow(t(2), 2) * p_AA(0)) - 5.9965918743193009e-28 * (pow(t(0), 2) * pow(t(2), 2) * p_AA(1)) - 2.2958363950258565e-43 * (pow(t(0), 2) * pow(t(2), 2) * p_AA(2)) + 1.2246467991473532e-16 * (pow(t(0

In [15]:
import operator
def clear_common_denominator(expr, denominator):
    #TODO
    "function makes no check that you have passed a valid common denominator actually"
    ctor, old_args = expr.Deconstruct()
    if ctor == sym._reduce_add:
        ret = 0
        for e in old_args:
            ret += clear_common_denominator(e)
        #call on each subexpression
        return ret
    if ctor == operator.truediv:
        #there is a denominator here
        ret = 1
    if ctor == sym._reduce_mul:
        ret = 1
        for e in old_args:
            #TODO traverse this tree
            pass


    assert isinstance(expr, sym.Expression), expr
    for old, new in rules:
        if expr.EqualTo(old):
            return new

    new_args = [_xreplace(e, rules) for e in old_args]
    return ctor(*new_args)
ctor, old_args = expr_in_t.Deconstruct()
print(old_args[5].Deconstruct())
# print(old_args[4].Deconstruct()[1][2].Deconstruct())

(<function _reduce_mul at 0x7fde277d4280>, [1.2246467991473532e-16, <Expression "p_AA(0)">, <Expression "((1 - pow(t(0), 2)) / (1 + pow(t(0), 2)))">, <Expression "((1 - pow(t(1), 2)) / (1 + pow(t(1), 2)))">])
